In [1]:
import pandas as pd
import numpy as np

from IPython.display import Audio
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

In [2]:
df= pd.read_csv('/kaggle/input/birdsss/xecanto_birds/metadata.csv')
df.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,file_name
0,abethr1,[],['song'],4.3906,38.2788,Turdus tephronotus,African Bare-eyed Thrush,Rolf A. de By,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/128013,abethr1/XC128013.ogg
1,abethr1,[],['call'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363501,abethr1/XC363501.ogg
2,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363502,abethr1/XC363502.ogg
3,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/363503,abethr1/XC363503.ogg
4,abethr1,[],"['call', 'song']",-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://www.xeno-canto.org/363504,abethr1/XC363504.ogg


In [3]:
len(df)

16941

In [4]:
label_counts = df['primary_label'].value_counts()

selected_labels = label_counts[label_counts >= 150].index
print(selected_labels)
len(selected_labels)
# Filter the original DataFrame
filtered_df = df[df['primary_label'].isin(selected_labels)]

Index(['barswa', 'wlwwar', 'thrnig1', 'eaywag1', 'comsan', 'woosan', 'combuz1',
       'eubeat1', 'hoopoe', 'cohmar1', 'litegr', 'combul2', 'rbsrob1',
       'blakit1', 'greegr', 'gnbcam2', 'rerswa1', 'somgre1', 'colsun2',
       'ratcis1', 'blbpuf2', 'categr', 'tafpri1', 'carcha1', 'egygoo'],
      dtype='object', name='primary_label')


In [5]:
filtered_df.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,file_name
895,barswa,[],"['call', 'flight call']",46.4605,6.3914,Hirundo rustica,Barn Swallow,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/113914,barswa/XC113914.ogg
896,barswa,[],['song'],35.0307,-120.6205,Hirundo rustica,Barn Swallow,Thomas G. Graves,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://www.xeno-canto.org/129647,barswa/XC129647.ogg
897,barswa,[],['song'],45.3675,-73.8566,Hirundo rustica,Barn Swallow,Patrick Turgeon,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/132406,barswa/XC132406.ogg
898,barswa,[],"['call', 'female', 'male', 'song']",56.1559,47.4939,Hirundo rustica,Barn Swallow,Albert Lastukhin,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/133096,barswa/XC133096.ogg
899,barswa,[],['song'],55.9937,-3.5605,Hirundo rustica,Barn Swallow,Mike Nelson,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/133802,barswa/XC133802.ogg


In [6]:
filtered_df['primary_label'].value_counts()


primary_label
barswa     500
comsan     500
wlwwar     500
thrnig1    500
eaywag1    500
woosan     486
combuz1    477
eubeat1    437
hoopoe     436
cohmar1    425
litegr     378
combul2    293
rbsrob1    281
blakit1    262
greegr     252
gnbcam2    239
rerswa1    227
somgre1    199
colsun2    181
ratcis1    172
categr     166
blbpuf2    166
tafpri1    161
carcha1    153
egygoo     152
Name: count, dtype: int64

In [7]:
path = '/kaggle/input/birdsss/xecanto_birds/'

In [8]:
sr =16000

In [9]:
example = filtered_df['file_name'].iloc[0]
example

'barswa/XC113914.ogg'

In [10]:
audio, _ = librosa.load(path + example, sr=sr)
Audio(audio, rate=sr)

In [11]:
def get_audio_duration(file_path, sr=sr):
  # Check if file exists before trying to load it
  if not os.path.exists(file_path):
    # print(f"Skipping file not found: {file_path}") # Print a message for skipped files
    return None  # Skip files that don't exist

  audio, _ = librosa.load(file_path, sr=sr)
  duration = librosa.get_duration(y=audio, sr=sr)
  return duration

In [12]:
filtered_df['duration'] = filtered_df['file_name'].apply(lambda x: get_audio_duration(os.path.join(path, x)))

Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC324914.ogg
Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC327328.ogg
Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC327364.ogg
Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC329025.ogg
Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC330211.ogg
Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC330429.ogg
Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC331493.ogg
Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC333650.ogg
Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC334130.ogg
Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC335495.ogg
Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC337180.ogg
Skipping file not found: /kaggle/input/birdsss/xecanto_birds/barswa/XC337979.ogg
Skipping file not found: /ka

/tmp/ipykernel_33/3532863766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['duration'] = filtered_df['file_name'].apply(lambda x: get_audio_duration(os.path.join(path, x)))


In [13]:
filtered_df = filtered_df.dropna(subset=['duration'])

# Reset index if needed
filtered_df.reset_index(drop=True, inplace=True)

# Print the filtered dataframe to verify the changes
print(filtered_df)

     primary_label        secondary_labels  \
0           barswa                      []   
1           barswa                      []   
2           barswa                      []   
3           barswa                      []   
4           barswa                      []   
...            ...                     ...   
2495        woosan                      []   
2496        woosan  ['afmdov1', 'eaywag1']   
2497        woosan                      []   
2498        woosan                      []   
2499        woosan                      []   

                                    type  latitude  longitude  \
0                ['call', 'flight call']   46.4605     6.3914   
1                               ['song']   35.0307  -120.6205   
2                               ['song']   45.3675   -73.8566   
3     ['call', 'female', 'male', 'song']   56.1559    47.4939   
4                               ['song']   55.9937    -3.5605   
...                                  ...       ...       

In [14]:
filtered_df['duration'].dtypes
filtered_df.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,file_name,duration
0,barswa,[],"['call', 'flight call']",46.4605,6.3914,Hirundo rustica,Barn Swallow,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/113914,barswa/XC113914.ogg,57.077562
1,barswa,[],['song'],35.0307,-120.6205,Hirundo rustica,Barn Swallow,Thomas G. Graves,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://www.xeno-canto.org/129647,barswa/XC129647.ogg,106.728000
2,barswa,[],['song'],45.3675,-73.8566,Hirundo rustica,Barn Swallow,Patrick Turgeon,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/132406,barswa/XC132406.ogg,10.788562
3,barswa,[],"['call', 'female', 'male', 'song']",56.1559,47.4939,Hirundo rustica,Barn Swallow,Albert Lastukhin,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/133096,barswa/XC133096.ogg,11.572250
4,barswa,[],['song'],55.9937,-3.5605,Hirundo rustica,Barn Swallow,Mike Nelson,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/133802,barswa/XC133802.ogg,15.490625


In [15]:
duplicates = filtered_df.duplicated(subset=['duration' , 'latitude' , 'type' , 'primary_label' , 'author'], keep=False)
duplicated_rows = filtered_df[duplicates]

display(len(duplicated_rows))
display(duplicated_rows)

12

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,file_name,duration
332,carcha1,[],"['call', 'song']",-34.0110,18.8078,Cossypha caffra,Cape Robin-Chat,Shannon Ronaldson,Creative Commons Attribution-NonCommercial-Sha...,1.0,https://www.xeno-canto.org/324664,carcha1/XC324664.ogg,10.004938
333,carcha1,[],"['call', 'song']",-34.0110,18.8078,Cossypha caffra,Cape Robin-Chat,Shannon Ronaldson,Creative Commons Attribution-NonCommercial-Sha...,1.0,https://www.xeno-canto.org/324665,carcha1/XC324665.ogg,10.004938
817,combuz1,[],['flight call'],55.2375,47.6393,Buteo buteo,Common Buzzard,Albert Lastukhin,Creative Commons Attribution-NonCommercial-Sha...,2.0,https://www.xeno-canto.org/144257,combuz1/XC144257.ogg,38.739625
818,combuz1,[],['flight call'],55.2375,47.6393,Buteo buteo,Common Buzzard,Albert Lastukhin,Creative Commons Attribution-NonCommercial-Sha...,2.0,https://www.xeno-canto.org/144258,combuz1/XC144258.ogg,38.739625
1650,litegr,[],['call'],36.8149,-6.3366,Egretta garzetta,Little Egret,José Carlos Sires,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/333878,litegr/XC333878.ogg,15.934688
1652,litegr,[],['call'],36.8149,-6.3366,Egretta garzetta,Little Egret,José Carlos Sires,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/333880,litegr/XC333880.ogg,17.397562
1655,litegr,[],['call'],36.8149,-6.3366,Egretta garzetta,Little Egret,José Carlos Sires,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/333884,litegr/XC333884.ogg,15.934688
1656,litegr,[],['call'],36.8149,-6.3366,Egretta garzetta,Little Egret,José Carlos Sires,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/333885,litegr/XC333885.ogg,17.397562
2087,somgre1,['combul2'],['song'],-6.7832,39.2033,Andropadus importunus,Sombre Greenbul,Louis A. Hansen,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/440353,somgre1/XC440353.ogg,130.680000
2095,somgre1,[],['song'],-6.7832,39.2033,Andropadus importunus,Sombre Greenbul,Louis A. Hansen,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/443103,somgre1/XC443103.ogg,130.680000


In [16]:
display(len(filtered_df))


2500

In [17]:
train_df, test_df = train_test_split(filtered_df, test_size=0.1, random_state=1234)

In [18]:
display(len(train_df))
display(test_df.head())

2000

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,file_name,duration
903,comsan,[],"['female', 'flight call', 'male', 'song']",52.3527,20.9197,Actitis hypoleucos,Common Sandpiper,Jarek Matusiak,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/129378,comsan/XC129378.ogg,11.363250
674,colsun2,[],['song'],-8.5110,35.9077,Hedydipna collaris,Collared Sunbird,Louis A. Hansen,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/516528,colsun2/XC516528.ogg,80.016000
1624,litegr,[],['call'],20.2864,100.0577,Egretta garzetta,Little Egret,Woraphot Bunkhwamdi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/305439,litegr/XC305439.ogg,64.862062
1214,eubeat1,['barswa'],['call'],39.4174,16.7033,Merops apiaster,European Bee-eater,Francesco Sottile,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/144124,eubeat1/XC144124.ogg,76.464000
2164,tafpri1,[],['song'],-2.5727,34.7597,Prinia subflava,Tawny-flanked Prinia,Bernard BOUSQUET,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/426074,tafpri1/XC426074.ogg,36.075000


In [19]:
display(len(test_df))
display(test_df.head())

500

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,file_name,duration
903,comsan,[],"['female', 'flight call', 'male', 'song']",52.3527,20.9197,Actitis hypoleucos,Common Sandpiper,Jarek Matusiak,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/129378,comsan/XC129378.ogg,11.363250
674,colsun2,[],['song'],-8.5110,35.9077,Hedydipna collaris,Collared Sunbird,Louis A. Hansen,Creative Commons Attribution-NonCommercial-Sha...,3.0,https://www.xeno-canto.org/516528,colsun2/XC516528.ogg,80.016000
1624,litegr,[],['call'],20.2864,100.0577,Egretta garzetta,Little Egret,Woraphot Bunkhwamdi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/305439,litegr/XC305439.ogg,64.862062
1214,eubeat1,['barswa'],['call'],39.4174,16.7033,Merops apiaster,European Bee-eater,Francesco Sottile,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/144124,eubeat1/XC144124.ogg,76.464000
2164,tafpri1,[],['song'],-2.5727,34.7597,Prinia subflava,Tawny-flanked Prinia,Bernard BOUSQUET,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/426074,tafpri1/XC426074.ogg,36.075000


In [20]:
drop_column = ['secondary_labels', 'author', 'license', 'url']

In [21]:
test_df.drop(drop_column, axis=1, inplace=True)

test_df.head()

,primary_label,type,latitude,longitude,scientific_name,common_name,rating,file_name,duration
903,comsan,"['female', 'flight call', 'male', 'song']",52.3527,20.9197,Actitis hypoleucos,Common Sandpiper,5.0,comsan/XC129378.ogg,11.363250
674,colsun2,['song'],-8.5110,35.9077,Hedydipna collaris,Collared Sunbird,3.0,colsun2/XC516528.ogg,80.016000
1624,litegr,['call'],20.2864,100.0577,Egretta garzetta,Little Egret,4.0,litegr/XC305439.ogg,64.862062
1214,eubeat1,['call'],39.4174,16.7033,Merops apiaster,European Bee-eater,3.5,eubeat1/XC144124.ogg,76.464000
2164,tafpri1,['song'],-2.5727,34.7597,Prinia subflava,Tawny-flanked Prinia,4.0,tafpri1/XC426074.ogg,36.075000


In [22]:
train_df.drop(drop_column, axis=1, inplace=True)

train_df.head()

,primary_label,type,latitude,longitude,scientific_name,common_name,rating,file_name,duration
1269,eubeat1,['flight call'],15.5442,73.7733,Merops apiaster,European Bee-eater,2.5,eubeat1/XC209554.ogg,12.146937
369,carcha1,['call'],-26.8243,26.6518,Cossypha caffra,Cape Robin-Chat,3.0,carcha1/XC457079.ogg,29.178812
744,combul2,['call'],0.6798,35.8208,Pycnonotus barbatus,Common Bulbul,5.0,combul2/XC211774.ogg,10.213875
1457,greegr,"['begging call', 'juvenile']",34.1262,-114.3193,Ardea alba,Great Egret,2.5,greegr/XC253851.ogg,87.196750
507,cohmar1,"['call', 'song']",54.5592,11.9695,Delichon urbicum,Common House-Martin,2.5,cohmar1/XC138515.ogg,25.704500


In [23]:
train_df['type'].value_counts().head(50)

type
['call']                                             609
['song']                                             432
['flight call']                                      233
['']                                                  88
['male', 'song']                                      85
['call', 'song']                                      81
['alarm call']                                        62
['call', 'flight call']                               50
['begging call', 'juvenile']                          29
['call', 'juvenile']                                  23
['alarm call', 'flight call']                         19
['call', 'male']                                      18
['call', 'female']                                    11
['flight call', 'male']                                9
['Song']                                               9
['call', 'male', 'song']                               8
['call', 'female', 'male']                             8
['juvenile']              

In [24]:
train_df.loc[train_df['type'].str.contains('call') & train_df['type'].str.contains('song'), 'type'] = 'both'
train_df.loc[train_df['type'].str.contains('call'), 'type'] = 'call'
train_df.loc[train_df['type'].str.contains('song'), 'type'] = 'song'
train_df.loc[train_df['type'] == "['']", 'type'] = 'blank'

In [25]:
train_df['type'].value_counts().head(50)

type
call                             1211
song                              551
both                              125
blank                              88
['Song']                            9
['juvenile']                        8
['Roosting place']                  1
['alarm near young']                1
['alarm']                           1
['barks', 'croaks', 'growls']       1
['Call']                            1
['male']                            1
['barks', 'croaks']                 1
['Coppulation', 'female']           1
Name: count, dtype: int64

In [26]:
train_df.loc[(train_df['type'] != 'call') & (train_df['type'] != 'song') & (train_df['type'] != 'both') & (train_df['type'] != 'blank'), 'type'] = 'blank'

In [27]:
train_df['type'].value_counts().head(50)

type
call     1211
song      551
both      125
blank     113
Name: count, dtype: int64

In [28]:
test_df['type'].value_counts().head(10)

type
['call']                        172
['song']                        114
['flight call']                  61
['male', 'song']                 23
['alarm call']                   16
['call', 'song']                 16
['']                             12
['call', 'flight call']          10
['begging call', 'juvenile']      6
['alarm call', 'call']            6
Name: count, dtype: int64

In [29]:
test_df.loc[test_df['type'].str.contains('call'), 'type'] = 'call'
test_df.loc[test_df['type'].str.contains('call') & test_df['type'].str.contains('song'), 'type'] = 'both'
test_df.loc[test_df['type'].str.contains('song'), 'type'] = 'song'
test_df.loc[test_df['type'] == "['']", 'type'] = 'blank'

In [30]:
test_df['type'].value_counts().head(10)

type
call             343
song             142
blank             12
['juvenile']       2
['dormitory']      1
Name: count, dtype: int64

In [31]:
test_df.loc[(test_df['type'] != 'call') & (test_df['type'] != 'song') & (test_df['type'] != 'both') & (test_df['type'] != 'blank'), 'type'] = 'blank'

In [32]:
test_df['type'].value_counts().head(10)

type
call     343
song     142
blank     15
Name: count, dtype: int64

DATA EXTRACTION

In [33]:
def extract_audio(filename, sr=16000):
  filepath = '/kaggle/input/birdsss/xecanto_birds/' + filename
  audio, _ = librosa.load(filepath, sr=sr)
  return audio

In [34]:
example = train_df['file_name'].iloc[0]

example

'eubeat1/XC209554.ogg'

In [35]:
example_audio = extract_audio(example)

example_audio

array([ 9.1968104e-06, -1.2148172e-05, -1.9846484e-06, ...,
       -1.7994751e-01, -1.6967215e-01, -1.8559830e-01], dtype=float32)

To convert to array from the audio

In [36]:
folder = example.split('/')[0]
folder

'eubeat1'

In [37]:
filename = example.split('/')[ 1]
filename

'XC209554.ogg'

In [38]:
new_filename = filename.replace('.ogg', '.npy')

new_filename

'XC209554.npy'

In [39]:
os.makedirs('/kaggle/working/train_npy', exist_ok=False)

In [40]:
save_dir = f'/kaggle/working/train_npy/{folder}/'
save_dir

'/kaggle/working/train_npy/eubeat1/'

In [41]:
os.makedirs(save_dir, exist_ok=False)

In [42]:
save_filename = f'{save_dir}{new_filename}'
save_filename

'/kaggle/working/train_npy/eubeat1/XC209554.npy'

In [43]:
with open(save_filename, 'w') as file:
  pass

np.save(save_filename, example_audio)

In [44]:
np.load(save_filename)

array([ 9.1968104e-06, -1.2148172e-05, -1.9846484e-06, ...,
       -1.7994751e-01, -1.6967215e-01, -1.8559830e-01], dtype=float32)

In [63]:

def save_audio_np(filenames, is_train):
    # Create directory to save the extracted np array
    if is_train:
        folder_dir = '/kaggle/working/train_npy'
    else:
        folder_dir = '/kaggle/working/test_npy'

    os.makedirs(folder_dir, exist_ok=True)  # Set exist_ok=True to avoid error if folder exists

    # List to store file paths
    filepaths = []

    # Loop over the filenames and extract the np objects and save to the respective directory
    for i, filename in enumerate(filenames):
        # Extract audio (hypothetical function)
        audio = extract_audio(filename)

        # Create new filepath
        folder = filename.split('/')[0]
        name = filename.split('/')[1]
        new_filename = name.replace('.ogg', '.npy')
        filepaths.append(f'{folder}/{new_filename}')

        # Create save directory
        save_dir = f'{folder_dir}/{folder}/'
        os.makedirs(save_dir, exist_ok=True)  # Set exist_ok=True here too

        # Create save filename
        save_filename = f'{save_dir}{new_filename}'

        # Save the .npy object
        np.save(save_filename, audio)

    # Assert that all filenames have been looped through and saved
    assert len(filenames) == len(filepaths)
    return filepaths


In [64]:
train_filenames = train_df['file_name']
train_filenames

1269    eubeat1/XC209554.ogg
369     carcha1/XC457079.ogg
744     combul2/XC211774.ogg
1457     greegr/XC253851.ogg
507     cohmar1/XC138515.ogg
                ...         
279     blbpuf2/XC504415.ogg
2041    somgre1/XC320617.ogg
664     colsun2/XC472395.ogg
1318    gnbcam2/XC174855.ogg
723      combul2/XC18013.ogg
Name: file_name, Length: 2000, dtype: object

In [65]:
train_filepaths = save_audio_np(train_filenames, is_train=True)
train_filepaths

['eubeat1/XC209554.npy',
 'carcha1/XC457079.npy',
 'combul2/XC211774.npy',
 'greegr/XC253851.npy',
 'cohmar1/XC138515.npy',
 'rbsrob1/XC288575.npy',
 'cohmar1/XC367979.npy',
 'carcha1/XC264285.npy',
 'greegr/XC145803.npy',
 'eaywag1/XC282377.npy',
 'thrnig1/XC145660.npy',
 'hoopoe/XC162592.npy',
 'rerswa1/XC145578.npy',
 'barswa/XC253981.npy',
 'carcha1/XC205903.npy',
 'rerswa1/XC194485.npy',
 'eubeat1/XC138232.npy',
 'barswa/XC263623.npy',
 'rbsrob1/XC388987.npy',
 'carcha1/XC515311.npy',
 'egygoo/XC331380.npy',
 'combul2/XC173036.npy',
 'thrnig1/XC180896.npy',
 'categr/XC438523.npy',
 'greegr/XC285219.npy',
 'tafpri1/XC246659.npy',
 'eubeat1/XC176059.npy',
 'blakit1/XC193181.npy',
 'hoopoe/XC153911.npy',
 'combuz1/XC170643.npy',
 'carcha1/XC322337.npy',
 'rerswa1/XC194483.npy',
 'somgre1/XC205863.npy',
 'combuz1/XC189733.npy',
 'categr/XC361546.npy',
 'cohmar1/XC193264.npy',
 'combul2/XC253034.npy',
 'combul2/XC207959.npy',
 'blbpuf2/XC492714.npy',
 'combul2/XC267457.npy',
 'combuz1/

In [69]:
example_before = extract_audio(train_df['file_name'].iloc[0])

display(example_before)
display(example_before.shape)

array([ 9.1968104e-06, -1.2148172e-05, -1.9846484e-06, ...,
       -1.7994751e-01, -1.6967215e-01, -1.8559830e-01], dtype=float32)

(194351,)

In [70]:
new_filename = train_df['file_name'].iloc[0].replace('.ogg', '.npy')
new_filename

'eubeat1/XC209554.npy'

In [71]:
example_after = np.load(f'/kaggle/working/train_npy/{new_filename}')

display(example_after)
display(example_after.shape)

array([ 9.1968104e-06, -1.2148172e-05, -1.9846484e-06, ...,
       -1.7994751e-01, -1.6967215e-01, -1.8559830e-01], dtype=float32)

(194351,)

In [72]:
np.array_equal(example_before, example_after)

True

In [73]:
train_df['filename_npy'] = train_filepaths

train_df.head()

,primary_label,type,latitude,longitude,scientific_name,common_name,rating,file_name,duration,filename_npy
1269,eubeat1,call,15.5442,73.7733,Merops apiaster,European Bee-eater,2.5,eubeat1/XC209554.ogg,12.146937,eubeat1/XC209554.npy
369,carcha1,call,-26.8243,26.6518,Cossypha caffra,Cape Robin-Chat,3.0,carcha1/XC457079.ogg,29.178812,carcha1/XC457079.npy
744,combul2,call,0.6798,35.8208,Pycnonotus barbatus,Common Bulbul,5.0,combul2/XC211774.ogg,10.213875,combul2/XC211774.npy
1457,greegr,call,34.1262,-114.3193,Ardea alba,Great Egret,2.5,greegr/XC253851.ogg,87.196750,greegr/XC253851.npy
507,cohmar1,both,54.5592,11.9695,Delichon urbicum,Common House-Martin,2.5,cohmar1/XC138515.ogg,25.704500,cohmar1/XC138515.npy


In [1]:
test_filenames = test_df['file_name']
test_filenames

NameError: name 'test_df' is not defined

In [ ]:
test_df['filename_npy'] = test_filepaths

test_df.head()

In [ ]:
test_filepaths = save_audio_np(test_filenames, is_train=False)

test_filepaths

In [ ]:
train_df.shape

In [ ]:
from sklearn.model_selection import train_test_split

# Get unique primary labels
unique_labels = train_df['primary_label'].unique()

# Create an empty DataFrame for the validation set
valid_df = pd.DataFrame()

# Iterate over unique labels and sample data for the validation set
for label in unique_labels:
    label_df = train_df[train_df['primary_label'] == label]
    _, valid_subset = train_test_split(label_df, test_size=0.1, random_state=42)
    valid_df = pd.concat([valid_df, valid_subset])

# Create the training set by excluding the validation data
final_train_df = train_df[~train_df.index.isin(valid_df.index)]

# Verify the split
print("Training set shape:", train_df.shape)
print("Validation set shape:", valid_df.shape)


In [ ]:
final_train_df['primary_label'].value_counts()


In [ ]:
valid_df['primary_label'].value_counts()

In [ ]:
final_train_df['set'] = 'train'
valid_df['set'] = 'val'

In [ ]:
final_train_df.head()

In [ ]:
valid_df.tail()

In [ ]:
 = pd.concat([final_train_df, valid_df])
combined_df.head()

In [ ]:
combined_df.tail()

5th part  

In [ ]:
import pickle
import matplotlib.pyplot as plt
import seaborn as sns